In [17]:
import pyAgrum.lib.notebook as gnb
import pyAgrum as gum
import cProfile
import random
import sys
import pyAgrum.lib.ipython as gnbipy
sys.path.append("C:\\Users\\zlhyl\\OneDrive\\Belgeler\\PhD")


    
from SDM_Functions import (
    update_decision_node_options, 
    add_nodes_from_student_to_advisor, add_nodes_from_advisor_to_student, add_equal_probabilities_for_new_option,
    add_decision_related_arcs_from_advisor_to_student,
    add_decision_related_arcs_from_student_to_advisor2, add_utility_related_arcs_from_advisor_to_student,
    add_utility_related_arcs_from_student_to_advisor, add_chance_related_arcs_from_advisor_to_student,
    add_chance_related_arcs_from_student_to_advisor, list_cpts_of_model, list_unmatched_node_cpts,
    add_noise_to_cpt, fill_student_cpts_with_noisy_data, fill_advisor_cpts_with_noisy_data,
    print_cpts_for_comparison, has_empty_cpts, print_all_cpts, print_all_utilitiesXX,
    get_utilities_as_dict_with_names,
    pool,selected_cpt_difficulty,
    selected_cpt_friends,selected_cpt_interest,add_cpt_from_pool,calculate_and_set_utilities_StudentS,calculate_and_set_utilities_AdvisorS,
    compare_decision_utilities,calculate_utilities_for_states,show_decisions,show_decision_utilities,compare_final_decisions,
    compare_final_decisions_panda,calculate_initial_results,print_all_utilitiesX,filter_chance_nodes,combine_influence_diagrams,
    add_nodes_with_percentage,show_decision_utilities3,
    get_utility_parents,find_matched_parents,find_unmatched_parents,get_parents_of_unmatched_parents,find_and_get_parents_of_unmatched_parents,
    check_unmatched_parents_in_expert_model,check_unmatched_parents_in_expert_model_final_situation,filter_chance_nodes,find_and_get_parents_of_matched_parents,
    get_parents_of_matched_parents,get_arcs_to_utility,delete_unmatched_arcs,remove_unwanted_arcs,transfer_utilities,filter_chance_nodes_ORJ
)

from advisor_model_v02 import create_advisor_model
from student_model_v03 import create_student_model


advisor_model = create_advisor_model()
student_model = create_student_model()


initial_advisor_model = create_advisor_model()
initial_student_model = create_student_model()


## ZY: TECHNICALLY WE ARE UPDATING THE ADVISOR MODEL, SINCE IT IS MORE EASY TO COMPLETE THE INTEGRATION, HOWEVER I PLAN TO CREATE AN EMPTY ID AND FILL IT USING TWO IDS.

## find unmatched parents of the utility node of the advisor model
unmatched_parents=find_unmatched_parents(advisor_model,student_model)

## add new states into the advisor decision node if there are some unmatched states with student model
advisor_model, advisor_decision_node_name = update_decision_node_options(advisor_model, student_model) 

## add new states into the student decision node if there are some unmatched states with advisor model
student_model, student_decision_node_name = update_decision_node_options(student_model, advisor_model) 

## update CPTs for new states of student model - ZY: THIS IS UNNECASSARY, SINCE WE ARE UPDATING THE ADVISOR MODEL
if random.random() < 0.5:
    add_equal_probabilities_for_new_option(student_model)
    function_used = "add_equal_probabilities_for_new_option"
else:
    add_cpt_from_pool(student_model, selected_cpt_difficulty, selected_cpt_friends, selected_cpt_interest)
    function_used = "add_cpt_from_pool"
        

## find matched parents of the utility node of the advisor model
matched_parents=find_matched_parents(student_model, advisor_model)

## Find unmatched arcs between the utility node and its parents in the advisor model, and delete them. Since we will use the utility function of the student
advisor_model=delete_unmatched_arcs(advisor_model, student_model)

## add nodes from student to advisor, this includes the parent nodes of the student utility node
advisor_model=add_nodes_from_student_to_advisor(advisor_model, student_model)
##add arcs to the advisor model
advisor_model=add_utility_related_arcs_from_student_to_advisor(student_model,advisor_model)
re_advisor_model=add_chance_related_arcs_from_student_to_advisor(student_model,advisor_model)
fin_advisor_model=remove_unwanted_arcs(re_advisor_model, initial_advisor_model,matched_parents)


## when we update the advisor model, copy the utilies from the student model
#***copy the utilities


utilities = {}

# Find the utility node in the advisor model
utility_node = next(node for node in fin_advisor_model.nodes() if fin_advisor_model.isUtilityNode(node))

# Set the name of the utility node
fin_advisor_model.variable(utility_node).setName("Student Utility")

# Iterate over all possible combinations of states
for int_val in ["yes", "no"]:
    for grade_val in [">C", "D", "F"]:
        for fr_val in ["yes", "no"]:
            for wr_val in ["high", "medium", "low"]:
                for cl_val in ["high", "medium", "low"]:
                        # Base value for having interest
                    u = 50 if int_val == "yes" else 5
    
                        # Incremental value for having friends
                    v = 60 if fr_val == "yes" else 10
    
                        # Value adjustments based on grade
                    if grade_val == ">C":
                       y = 50
                    elif grade_val == "D":
                       y = 30
                    else:
                       y = 0
    
                        # Value adjustments based on workload
                    if wr_val == "high":
                       x = 20
                    elif wr_val == "medium":
                       x = 30
                    else:
                       x = 40
    
                        # Value adjustments based on workload
                    if cl_val == "high":
                       z = 20
                    elif cl_val == "medium":
                       z = 30
                    else:
                       z = 40
    
                        # Store the utility value for the combination of states
                    utilities[(int_val, grade_val, fr_val, wr_val, cl_val)] = 0.1*u + 0.2*v + 0.5*y + 0.1*x + 0.1*z
    
    # Fill the utility node with the computed values
for int_val in ["yes", "no"]:
    for grade_val in [">C", "D", "F"]:
        for fr_val in ["yes", "no"]:
            for wr_val in ["high", "medium", "low"]:
                for cl_val in ["high", "medium", "low"]:
                       # Set the utility value using the generated utilities
                    fin_advisor_model.utility(utility_node)[{'interest': int_val, 'Grade': grade_val, 'friends': fr_val, 'workload': wr_val, 'classsize': cl_val}] = utilities[(int_val, grade_val, fr_val, wr_val, cl_val)]


## delete unmatched parents of the advisor utility node
combined_model=filter_chance_nodes(fin_advisor_model,unmatched_parents)

gnb.sideBySide(combined_model)
        

